In [1]:
import numpy as np
import keras
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import utils
import imblearn
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from stop_words import get_stop_words

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
NAME_TRAIN = r'C:\Users\Annie\Documents\Working\Корпус биографических текстов\Данные для обучения сетей\oversampling\ALL.csv'
NAME_TEST = r'C:\Users\Annie\Documents\Working\Корпус биографических текстов\Данные для обучения сетей\oversampling\TEST.csv'
EPOCHS = 2
BATCH_SIZE = 16
HIDDEN_NEURONS = 1024
LAYERS = 2
ACTIVATION = "tanh"
DROPOUT = 0.5
EARLY_STOPPING = 5
MODEL_PATH = "1.hdf5"
GAMMA = 0.1
N_NEIGH = 4

In [3]:
import random

with open(NAME_TRAIN) as f:
        df = pd.read_csv(f, sep=';', header=None, decimal = '.')

train_data = df.values
random.seed(139)  # для воспроизводимость
random.shuffle(train_data)
m = len(train_data)

train_length = int(0.8 * m)
train_data, test_data = train_data[:train_length],\
    train_data[train_length:]

In [4]:
df = pd.DataFrame(train_data)
texts = df[0].tolist()
categories = df[1].tolist()
vectors = df.drop([0,1], axis=1).values

df = pd.DataFrame(test_data)
texts_test = df[0].tolist()
categories_test = df[1].tolist()
vectors_test = df.drop([0,1], axis=1).values

## Text import

In [20]:
#не запускать
with open(NAME_TRAIN) as f:
        df = pd.read_csv(f, sep=';', header=None, decimal = '.')
with open(NAME_TEST) as f:
        df_test = pd.read_csv(f, sep=';', header=None, decimal = '.')
texts = df[0].tolist()
categories = df[1].tolist()
vectors = df.drop([0,1], axis=1).values
texts_test = df_test[0].tolist()
categories_test = df_test[1].tolist()
vectors_test = df_test.drop([0,1], axis=1).values

## Preprocessing

In [5]:
types_of_facts  = {"birth": 0, "death": 1, "occupation": 2, "education": 3, "affiliation": 4, "family": 5, \
         "parenting": 6, "professional_events": 7, "personal_events": 8, "residence": 9}

print("training sample:")
num_classes = len(types_of_facts)
count = np.zeros(shape=(num_classes))
for i in range(len(categories)):
    categories[i] = types_of_facts[categories[i]]
    count[categories[i]]+=1
print(count)
print("count of sentences: {0}".format(count.sum()))
M=count.max()
print("max class: {0}".format(M))

print("test sample:")
count_test = np.zeros(shape=(num_classes))
for i in range(len(categories_test)):
    categories_test[i] = types_of_facts[categories_test[i]]
    count_test[categories_test[i]]+=1
print(count_test)

training sample:
[191. 165. 319. 516. 171.  44.  15. 118.  23.  15.]
count of sentences: 1577.0
max class: 516.0
test sample:
[ 26.  21. 127.  67.  29.   6.   8.  81.  15.  15.]


In [75]:
print(vectors)

[[0.08885499509051442 -0.07851399946957828 0.2726290060672909 ...
  -0.12033999804407358 0.1860189912840724 0.3426859998144209]
 [0.08885499509051442 -0.07851399946957828 0.2726290060672909 ...
  -0.12033999804407358 0.1860189912840724 0.3426859998144209]
 [0.15586599835660309 -0.004857996478676796 0.06730599910952151 ...
  -0.2980579999275505 0.5592820029705763 -0.31733799912035465]
 ...
 [0.09618299768771976 -0.061186004430055625 0.1815540031529963 ...
  -0.4477570094168186 0.10406100004911424 -0.17310800217092034]
 [0.015114001929759981 0.08529199712211266 0.18716900423169136 ...
  -0.3612539973109961 0.44122800789773453 0.0526020023971796]
 [0.2685830066911876 -0.1521739938762039 0.3618329963646829 ...
  -0.18147299624979496 0.6014230037108064 -0.060259000398218625]]


In [39]:
print(types_of_facts['birth'])

0


In [6]:
morph = pymorphy2.MorphAnalyzer()

def transform_text(text):
    text = text.lower()
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    text = ' '.join(morph.parse(word)[0].normal_form for word in text.split())
    return text

for i in range(len(texts)):
    texts[i] = transform_text(texts[i])
for i in range(len(texts_test)):
    texts_test[i] = transform_text(texts_test[i])

In [34]:
#sampling word2vec Xu Chen Word Embedding Composition for Data Imbalances in Sentiment and Emotion Classification
import random
import copy

def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

for type_fact in types_of_facts:
    #new_cat=[]
    new_vec=[]
    count=0
    print(type_fact)
    for i in range(len(categories)):
        if get_key(types_of_facts,categories[i])==type_fact:
            count+=1
    new_vec=np.zeros(shape=(count,300))
    k=0
    for i in range(len(categories)):
        if get_key(types_of_facts,categories[i])==type_fact:
            new_vec[k]=vectors[i]
            k+=1    
    
    new_count=count
    #while count<516:
    while count<sampling_strategy[types_of_facts[type_fact]]:
        alpha=random.random()
        num1=random.randint(0,new_count-1)
        num2=random.randint(0,new_count-1)
        new_elem=np.zeros(shape=(1,300))
        for j in range(len(new_vec[num1])):
            new_elem[0][j]=new_vec[num1][j]-alpha*new_vec[num2][j]

        vectors= np.append(vectors,new_elem,axis=0)

        categories=np.append(categories,types_of_facts[type_fact])
        count+=1


birth
death
occupation
education
affiliation
family
parenting
professional_events
personal_events
residence


In [105]:
len(categories)

5160

In [35]:
num_classes = len(types_of_facts)
count = np.zeros(shape=(num_classes))
for i in range(len(categories)):
    count[categories[i]]+=1
print(count)
print("count of sentences: {0}".format(count.sum()))
M=count.max()
print("max class: {0}".format(M))

[354. 340. 418. 516. 344. 280. 266. 317. 270. 266.]
count of sentences: 3371.0
max class: 516.0


In [7]:
sw_list = get_stop_words('russian')+get_stop_words('english')

vectorizer = CountVectorizer(stop_words=sw_list)
all_words = vectorizer.fit_transform(texts)
bag_of_words = all_words.toarray()
all_words = vectorizer.transform(texts_test)
bag_of_words_test = all_words.toarray()

vectorizer = TfidfVectorizer(stop_words=sw_list)
all_words = vectorizer.fit_transform(texts)
tfidf = all_words.toarray()
all_words = vectorizer.transform(texts_test)
tfidf_test = all_words.toarray()

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [33]:
k_parameter = 0.5

sampling_strategy = {0: int(round((M-count[0])*k_parameter+count[0])), 1: int(round((M-count[1])*k_parameter+count[1])), 2: int(round((M-count[2])*k_parameter+count[2])), \
                     3: int(round((M-count[3])*k_parameter+count[3])), 4: int(round((M-count[4])*k_parameter+count[4])), 5: int(round((M-count[5])*k_parameter+count[5])), \
                     6: int(round((M-count[6])*k_parameter+count[6])), 7: int(round((M-count[7])*k_parameter+count[7])), 8: int(round((M-count[8])*k_parameter+count[8])), \
                     9: int(round((M-count[9])*k_parameter+count[9]))}

print(sampling_strategy)

{0: 354, 1: 340, 2: 418, 3: 516, 4: 344, 5: 280, 6: 266, 7: 317, 8: 270, 9: 266}


In [39]:
sampling_strategy = 'auto'

## Random Oversampling

In [193]:
ros = RandomOverSampler(random_state=42, sampling_strategy=sampling_strategy)
#vectors_ros, categories_ros = ros.fit_resample(bag_of_words, categories)
#vectors_ros, categories_ros = ros.fit_resample(tfidf, categories)
vectors_ros, categories_ros = ros.fit_resample(vectors, categories)
print(len(vectors_ros))

4265


## SMOTE

In [194]:
sm = SMOTE(random_state=42, sampling_strategy=sampling_strategy)
#vectors_sm, categories_sm = sm.fit_resample(bag_of_words, categories)
#vectors_sm, categories_sm = sm.fit_resample(tfidf, categories)
vectors_sm, categories_sm = sm.fit_resample(vectors, categories)
print(len(vectors_sm))

4265


## BorderlineSMOTE

In [195]:
bsm = BorderlineSMOTE(random_state=42, sampling_strategy=sampling_strategy)
#vectors_bsm, categories_bsm = bsm.fit_resample(bag_of_words, categories)
#vectors_bsm, categories_bsm = bsm.fit_resample(tfidf, categories)
vectors_bsm, categories_bsm = bsm.fit_resample(vectors, categories)
print(len(vectors_bsm))

4265


## ADASYN

In [196]:
adasyn = ADASYN(random_state=42, sampling_strategy=sampling_strategy)
#vectors_as, categories_as = adasyn.fit_resample(bag_of_words, categories)
#vectors_as, categories_as = adasyn.fit_resample(tfidf, categories)
vectors_as, categories_as = adasyn.fit_resample(vectors, categories)
print(len(vectors_as))

4293


## KNN

In [36]:
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(vectors, categories) 
predict = neigh.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))
print(f1_score(predict, categories_test, average=None))
print(precision_score(predict, categories_test, average=None))
print(recall_score(predict, categories_test, average=None))

Accuracy: 65.063291
F1-score: 60.150465
Precision: 69.249918
Recall: 59.182503
[0.75       0.76       0.6728972  0.74853801 0.69047619 0.66666667
 0.27272727 0.47368421 0.46153846 0.51851852]
[0.92307692 0.9047619  0.56692913 0.95522388 1.         1.
 0.375      0.33333333 0.4        0.46666667]
[0.63157895 0.65517241 0.82758621 0.61538462 0.52727273 0.5
 0.21428571 0.81818182 0.54545455 0.58333333]


In [197]:
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(vectors_ros, categories_ros) 
predict = neigh.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))
print(f1_score(predict, categories_test, average=None))
print(precision_score(predict, categories_test, average=None))
print(recall_score(predict, categories_test, average=None))

neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(vectors_sm, categories_sm) 
predict = neigh.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))
print(f1_score(predict, categories_test, average=None))
print(precision_score(predict, categories_test, average=None))
print(recall_score(predict, categories_test, average=None))

neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(vectors_bsm, categories_bsm) 
predict = neigh.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))
print(f1_score(predict, categories_test, average=None))
print(precision_score(predict, categories_test, average=None))
print(recall_score(predict, categories_test, average=None))

neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(vectors_as, categories_as) 
predict = neigh.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))
print(f1_score(predict, categories_test, average=None))
print(precision_score(predict, categories_test, average=None))
print(recall_score(predict, categories_test, average=None))

Accuracy: 71.392405
F1-score: 66.936215
Precision: 71.917279
Recall: 68.005393
[0.77419355 0.85714286 0.74235808 0.79012346 0.75324675 0.75
 0.5        0.57142857 0.41666667 0.53846154]
[0.92307692 1.         0.66929134 0.95522388 1.         1.
 0.375      0.4691358  0.33333333 0.46666667]
[0.66666667 0.75       0.83333333 0.67368421 0.60416667 0.6
 0.75       0.73076923 0.55555556 0.63636364]
Accuracy: 71.645570
F1-score: 66.368617
Precision: 73.821294
Recall: 63.904952
[0.83333333 0.88888889 0.7281106  0.82894737 0.75324675 0.66666667
 0.46153846 0.62318841 0.35294118 0.5       ]
[0.96153846 0.95238095 0.62204724 0.94029851 1.         1.
 0.375      0.5308642  0.4        0.6       ]
[0.73529412 0.83333333 0.87777778 0.74117647 0.60416667 0.5
 0.6        0.75438596 0.31578947 0.42857143]
Accuracy: 68.354430
F1-score: 63.993207
Precision: 70.795378
Recall: 67.168878
[0.78688525 0.82608696 0.68493151 0.84       0.64285714 0.57142857
 0.54545455 0.54929577 0.38095238 0.57142857]
[0.92307

# SVM

In [37]:
#clf = svm.LinearSVC()
clf = svm.SVC(gamma = GAMMA)
clf.fit(vectors, categories) 
predict = clf.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100))    

Accuracy: 81.772152
F1-score: 76.946060
Precision: 75.630705
Recall: 87.110306


In [199]:
clf = svm.SVC(gamma = GAMMA)
clf.fit(vectors_sm, categories_sm) 
predict = clf.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100)) 

Accuracy: 83.037975
F1-score: 79.249732
Precision: 76.374927
Recall: 90.194547


In [200]:
clf = svm.SVC(gamma = GAMMA)
clf.fit(vectors_bsm, categories_bsm) 
predict = clf.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100)) 

Accuracy: 82.025316
F1-score: 77.752071
Precision: 74.452982
Recall: 90.725933


In [205]:
clf = svm.SVC(gamma = GAMMA)
clf.fit(vectors_as, categories_as) 
predict = clf.predict(vectors_test)
print('Accuracy: %f' % (accuracy_score(predict, categories_test)*100))
print('F1-score: %f' % (f1_score(predict, categories_test, average="macro")*100))
print('Precision: %f' % (precision_score(predict, categories_test, average="macro")*100))
print('Recall: %f' % (recall_score(predict, categories_test, average="macro")*100)) 

Accuracy: 83.797468
F1-score: 79.619753
Precision: 76.434494
Recall: 91.319114


## Feedforward network

In [8]:
x_train = bag_of_words
y_train = keras.utils.to_categorical(categories, num_classes)
x_test = bag_of_words_test
y_test = keras.utils.to_categorical(categories_test, num_classes)

In [9]:
model = Sequential()
model.add(Dense(HIDDEN_NEURONS, input_dim=len(x_train[0]), activation=ACTIVATION))
model.add(Dropout(DROPOUT))
for i in range(LAYERS):
    model.add(Dense(HIDDEN_NEURONS, activation=ACTIVATION))
    model.add(Dropout(DROPOUT))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

callbacks = []
callbacks.append(EarlyStopping(monitor="val_acc", patience=EARLY_STOPPING))
model_checkpoint = ModelCheckpoint(filepath=MODEL_PATH, monitor="val_acc",
                                       save_best_only=True, save_weights_only=True)
callbacks.append(model_checkpoint)
model.fit(x_train, y_train, epochs=2, batch_size = BATCH_SIZE, validation_data=(x_test, y_test), callbacks=callbacks)

predict = np.argmax(model.predict(x_test), axis=1)
answer = np.argmax(y_test, axis=1)
print('Accuracy: %f' % (accuracy_score(predict, answer)*100))
print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))
print('Precision: %f' % (precision_score(predict, answer, average="macro")*100))
print('Recall: %f' % (recall_score(predict, answer, average="macro")*100))    

Train on 1577 samples, validate on 395 samples
Epoch 1/2
1577/1577 [==============================] - 25s 16ms/step - loss: 0.7350 - acc: 0.7806 - val_loss: 0.8060 - val_acc: 0.7671
Epoch 2/2
1577/1577 [==============================] - 24s 15ms/step - loss: 0.1248 - acc: 0.9645 - val_loss: 0.9984 - val_acc: 0.8000
Accuracy: 80.000000
F1-score: 73.625253
Precision: 72.593262
Recall: 82.004576


In [10]:
model.fit(x_train, y_train, epochs=1, batch_size = BATCH_SIZE, validation_data=(x_test, y_test), callbacks=callbacks)

predict = np.argmax(model.predict(x_test), axis=1)
answer = np.argmax(y_test, axis=1)
print('Accuracy: %f' % (accuracy_score(predict, answer)*100))
print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))
print('Precision: %f' % (precision_score(predict, answer, average="macro")*100))
print('Recall: %f' % (recall_score(predict, answer, average="macro")*100))  

Train on 1577 samples, validate on 395 samples
Epoch 1/1
1577/1577 [==============================] - 25s 16ms/step - loss: 0.0135 - acc: 0.9949 - val_loss: 1.2027 - val_acc: 0.7747
Accuracy: 77.468354
F1-score: 75.504739
Precision: 76.414882
Recall: 77.075376


In [120]:
print(len(vectors_ros))

4265
